### seq2seq with attention code를 하나씩 뜯어보기
- 본 코드는 유투버 나동빈 님의 seq2seq 논문 재현 코드입니다.
- 오리지널 코드에도 자세하게 달려있으나 제가 하나씩 뜯어보면서 이해한 부분들을 조금 더 상세하게 적어놓았으니 참고하면 좋을 것 같습니다.
- original code link: https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Sequence_to_Sequence_with_Attention_Tutorial.ipynb

In [1]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [2]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)
spacy_de = spacy.load('de_core_news_sm')

In [3]:
# 간단히 토큰화(tokenization) 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


In [4]:
# 독일어(Deutsch) 문장을 토큰화 하는 함수 (순서를 뒤집지 않음)
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

# 영어(English) 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

In [38]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True)

In [39]:
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

In [40]:
"""Field 객체의 build_vocab 메소드를 사용하여 영어와 독어의 단어 사전을 생성한다.
min_freq를 사용하여 최소 등장 단어의 개수를 정의한다."""
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"SRC 단어 사전의 총 개수: len(SRC): {len(SRC.vocab)}")
print(f"TRG 단어 사전의 총 개수: len(TRG): {len(TRG.vocab)}")

SRC 단어 사전의 총 개수: len(SRC): 7853
TRG 단어 사전의 총 개수: len(TRG): 5893


In [41]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


In [42]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

BATCH_SIZE = 128

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device=device)

cuda


# 모델 아키텍처

## 인코더 아키텍처
- 주어진 소스 문장을 문맥 벡터(context vector)로 인코딩한다.
- 하이퍼 파라미터
    - input_dim: 하나의 다넝에 대한 원핫인코딩 차원
    - embed_dim: 임베딩 차원
    - enc_hidden_dim: 인코더의 hidden_state 차원
    - dec_hidden_dim: 디코더의 hidden_state 차원
    - dropout_ratio: 드롭아웃 비율

In [43]:
import torch.nn as nn

# 인코더(Encoder) 아키텍처 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, enc_hidden_dim, dec_hidden_dim, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding)을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # 양방향(bidirectional) GRU 레이어
        self.rnn = nn.GRU(embed_dim, enc_hidden_dim, bidirectional=True)

        # FC 레이어
        self.fc = nn.Linear(enc_hidden_dim * 2, dec_hidden_dim)

        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더는 소스 문장을 입력으로 받아 문맥 벡터(context vector)를 반환        
    def forward(self, src):
        # src: [단어 개수, 배치 크기]: 각 단어의 인덱스(index) 정보
        embedded = self.dropout(self.embedding(src))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        outputs, hidden = self.rnn(embedded)
        # outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
        # hidden: [레이어 개수 * 방향의 수, 배치 크기, 인코더 히든 차원]: 현재까지의 모든 단어의 정보

        # hidden은 [forward_1, backward_1, forward_2, backward_2, ...] 형태로 구성
        # 따라서 hidden[-2, :, :]은 forwards의 마지막 값
        # 따라서 hidden[-1, :, :]은 backwards의 마지막 값
        # 디코더(decoder)의 첫 번째 hidden (context) vector는 인코더의 마지막 hidden을 이용
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))

        # outputs은 Attention 목적으로, hidden은 context vector 목적으로 사용
        return outputs, hidden

# 디코더 아키텍처

In [44]:
# 디코더(Decoder) 아키텍처 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, enc_hidden_dim, dec_hidden_dim, dropout_ratio, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding) 말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # GRU 레이어
        self.rnn = nn.GRU((enc_hidden_dim * 2) + embed_dim, dec_hidden_dim)

        # FC 레이어
        self.fc_out = nn.Linear((enc_hidden_dim * 2) + dec_hidden_dim + embed_dim, output_dim)
        
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환     
    def forward(self, input, hidden, enc_outputs):
        # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
        # hidden: [배치 크기, 히든 차원]
        # enc_outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
        input = input.unsqueeze(0)
        # input: [단어 개수 = 1, 배치 크기]

        embedded = self.dropout(self.embedding(input))
        # embedded: [단어 개수 = 1, 배치 크기, 임베딩 차원]

        attention = self.attention(hidden, enc_outputs)
        # attention: [배치 크기, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들
        attention = attention.unsqueeze(1)
        # attention: [배치 크기, 1, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들

        enc_outputs = enc_outputs.permute(1, 0, 2)
        # enc_outputs: [배치 크기, 단어 개수, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보

        weighted = torch.bmm(attention, enc_outputs) # 행렬 곱 함수
        # weighted: [배치 크기, 1, 인코더 히든 차원 * 방향의 수]

        weighted = weighted.permute(1, 0, 2)
        # weighted: [1, 배치 크기, 인코더 히든 차원 * 방향의 수]
        
        rnn_input = torch.cat((embedded, weighted), dim=2)
        # rnn_input: [1, 배치 크기, 인코더 히든 차원 * 방향의 수 + embed_dim]: 어텐션이 적용된 현재 단어 입력 정보
        
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        # output: [단어 개수, 배치 크기, 디코더 히든 차원 * 방향의 수]
        # hidden: [레이어 개수 * 방향의 수, 배치 크기, 디코더 히든 차원]: 현재까지의 모든 단어의 정보

        # 현재 예제에서는 단어 개수, 레이어 개수, 방향의 수 모두 1의 값을 가짐
        # 따라서 output: [1, 배치 크기, 디코더 히든 차원], hidden: [1, 배치 크기, 디코더 히든 차원]
        # 다시 말해 output과 hidden의 값 또한 동일
        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        # prediction = [배치 크기, 출력 차원]
        
        # (현재 출력 단어, 현재까지의 모든 단어의 정보)
        return prediction, hidden.squeeze(0)

# 어텐션 아키텍처
- 하나의 어텐션은 인코더의 전체 토큰에 대한 출력을 입력으로 받는 FC의 파라미터를 공유하여 사용한다.
- (전체 인코더 출력+현재 디코더의 히든) -> 디코더의 히든 = 실제 Attention 값
- 하이퍼 파라미터
    - enc_hidden_dim: 인코더의 hidden_state 차원
    - dec_hidden_dim: 디코더의 hidden_state 차원

In [45]:
import torch.nn.functional as F

# 어텐션(Attention) 아키텍처 정의
class Attention(nn.Module):
    def __init__(self, enc_hidden_dim, dec_hidden_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hidden_dim * 2) + dec_hidden_dim, dec_hidden_dim)
        self.v = nn.Linear(dec_hidden_dim, 1, bias=False)

    def forward(self, hidden, enc_outputs):
        # hidden: [배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # enc_outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
        batch_size = enc_outputs.shape[1]
        src_len = enc_outputs.shape[0]

        # 현재 디코더의 히든 상태(hidden state)를 src_len만큼 반복
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        enc_outputs = enc_outputs.permute(1, 0, 2)
        # hidden: [배치 크기, 단어 개수, 디코더 히든 차원]: 현재까지의 모든 단어의 정보
        # enc_outputs: [배치 크기, 단어 개수, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보

        energy = torch.tanh(self.attn(torch.cat((hidden, enc_outputs), dim=2)))
        # energy: [배치 크기, 단어 개수, 디코더 히든 차원]

        attention = self.v(energy).squeeze(2)
        # attention: [배치 크기, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들

        return F.softmax(attention, dim=1)

#### **Seq2Seq with Attention 아키텍처**

* 앞서 정의한 인코더(encoder)와 디코더(decoder)를 가지고 있는 하나의 아키텍처입니다.
    * **인코더(encoder)**: 주어진 소스 문장을 문맥 벡터(context vector)로 인코딩합니다.
    * **디코더(decoder)**: 주어진 문맥 벡터(context vector)를 타겟 문장으로 디코딩합니다.
    * 단, **디코더는 한 단어씩** 넣어서 한 번씩 결과를 구합니다.
    * 또한 디코더는 문맥 벡터 뿐만 아니라 <b>인코더의 모든 출력을 참고하여 어텐션(attention)을 진행</b>합니다.
* **Teacher forcing**: 디코더의 예측(prediction)을 다음 입력으로 사용하지 않고, 실제 목표 출력(ground-truth)을 다음 입력으로 사용하는 기법

In [46]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]
        # 먼저 인코더를 거쳐 전체 출력과 문맥 벡터(context vector)를 추출
        enc_outputs, hidden = self.encoder(src)

        # 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫 번째 입력은 항상 <sos> 토큰
        input = trg[0, :]

        # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, enc_outputs)

            outputs[t] = output # FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기

        return outputs

#### **학습(Training)**

* 하이퍼 파라미터 설정 및 모델 초기화

In [47]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
ENCODER_HIDDEN_DIM = 512
DECODER_HIDDEN_DIM = 512
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [48]:
# 어텐션(attention) 객체 선언
attn = Attention(ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM)

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, DEC_DROPOUT_RATIO, attn)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

* **모델 가중치 파라미터 초기화**

In [49]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

* 학습 및 평가 함수 정의
    * 기본적인 Seq2Seq 모델과 완전히 동일하게 작성할 수 있습니다.

In [50]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [51]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0
    
    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()

        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기, 출력 차원]
        output_dim = output.shape[-1]
        
        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어의 개수 - 1) * batch size, output dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어의 개수 - 1) * batch size]
        
        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산
        
        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # 파라미터 업데이트
        optimizer.step()
        
        # 전체 손실 값 계산
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [52]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0
    
    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]
            
            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].view(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

* 학습(training) 및 검증(validation) 진행
    * **학습 횟수(epoch)**: 10

In [53]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [54]:
import time
import math
import random

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq_with_attention.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 1m 9s
	Train Loss: 5.035 | Train PPL: 153.691
	Validation Loss: 4.840 | Validation PPL: 126.508
Epoch: 02 | Time: 1m 10s
	Train Loss: 4.167 | Train PPL: 64.547
	Validation Loss: 4.504 | Validation PPL: 90.376
Epoch: 03 | Time: 1m 6s
	Train Loss: 3.524 | Train PPL: 33.911
	Validation Loss: 3.798 | Validation PPL: 44.617
Epoch: 04 | Time: 1m 7s
	Train Loss: 2.953 | Train PPL: 19.157
	Validation Loss: 3.402 | Validation PPL: 30.021
Epoch: 05 | Time: 1m 8s
	Train Loss: 2.536 | Train PPL: 12.626
	Validation Loss: 3.224 | Validation PPL: 25.125
Epoch: 06 | Time: 1m 8s
	Train Loss: 2.233 | Train PPL: 9.324
	Validation Loss: 3.141 | Validation PPL: 23.125
Epoch: 07 | Time: 1m 7s
	Train Loss: 1.965 | Train PPL: 7.137
	Validation Loss: 3.197 | Validation PPL: 24.451
Epoch: 08 | Time: 1m 7s
	Train Loss: 1.781 | Train PPL: 5.938
	Validation Loss: 3.185 | Validation PPL: 24.175
Epoch: 09 | Time: 1m 7s
	Train Loss: 1.606 | Train PPL: 4.982
	Validation Loss: 3.228 | Validation PPL: 

In [55]:
# # 학습된 모델 저장
# from google.colab import files

# files.download('seq2seq_with_attention.pt')
torch.save(model, './' + 'seq2seq_with_attention.pt') 

#### **모델 최종 테스트(testing) 결과 확인**

In [58]:
model = Seq2Seq(enc, dec, device).to(device)
model = torch.load('seq2seq_with_attention.pt')

test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 3.266 | Test PPL: 26.215


#### **나만의 데이터로 모델 사용해보기**

In [59]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        enc_outputs, hidden = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, enc_outputs)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [60]:
example_idx = 10

src = vars(test_dataset.examples[example_idx])['src']
trg = vars(test_dataset.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['eine', 'mutter', 'und', 'ihr', 'kleiner', 'sohn', 'genießen', 'einen', 'schönen', 'tag', 'im', 'freien', '.']
타겟 문장: ['a', 'mother', 'and', 'her', 'young', 'song', 'enjoying', 'a', 'beautiful', 'day', 'outside', '.']
전체 소스 토큰: ['<sos>', 'eine', 'mutter', 'und', 'ihr', 'kleiner', 'sohn', 'genießen', 'einen', 'schönen', 'tag', 'im', 'freien', '.', '<eos>']
소스 문장 인덱스: [2, 8, 364, 10, 134, 70, 624, 565, 19, 780, 200, 20, 88, 4, 3]
모델 출력 결과: a mom and her son enjoying enjoying a beautiful day . <eos>


In [61]:
src = tokenize_de("Guten Abend.")

print(f'소스 문장: {src}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['Guten', 'Abend', '.']
전체 소스 토큰: ['<sos>', 'guten', 'abend', '.', '<eos>']
소스 문장 인덱스: [2, 3798, 1163, 4, 3]
모델 출력 결과: chinese workers are lanterns at night . <eos>


--------

# 코드 뜯어보기

### batch 데이터 확인

In [ ]:
for i, batch in enumerate(train_iterator):
    first_batch_src = batch.src
    first_batch_trg = batch.trg

    print(f"첫 번째 배치 크기: {first_batch_src.shape}")
    print()
    print('첫 번째 문장은 모두 <sos> 토큰으로 이루어져 있다')
    print(first_batch_src)
    print()

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(first_batch_src.shape[0]):
        print(f"인덱스 {i}: {first_batch_src[i][0].item()}")

    # 첫 번째 배치만 확인
        # 여기서 첫 번째 배치란 1열 모두를 의미한다. tensor의 shape이 (30, 128)로 되어있기 때문
    break

첫 번째 배치 크기: torch.Size([128, 28])

첫 번째 문장은 모두 <sos> 토큰으로 이루어져 있다
tensor([[ 2, 76,  0,  ...,  1,  1,  1],
        [ 2,  5, 13,  ...,  1,  1,  1],
        [ 2, 43, 41,  ...,  1,  1,  1],
        ...,
        [ 2,  5, 13,  ...,  1,  1,  1],
        [ 2,  5, 70,  ...,  1,  1,  1],
        [ 2,  8, 36,  ...,  1,  1,  1]], device='cuda:0')

인덱스 0: 2
인덱스 1: 2
인덱스 2: 2
인덱스 3: 2
인덱스 4: 2
인덱스 5: 2
인덱스 6: 2
인덱스 7: 2
인덱스 8: 2
인덱스 9: 2
인덱스 10: 2
인덱스 11: 2
인덱스 12: 2
인덱스 13: 2
인덱스 14: 2
인덱스 15: 2
인덱스 16: 2
인덱스 17: 2
인덱스 18: 2
인덱스 19: 2
인덱스 20: 2
인덱스 21: 2
인덱스 22: 2
인덱스 23: 2
인덱스 24: 2
인덱스 25: 2
인덱스 26: 2
인덱스 27: 2
인덱스 28: 2
인덱스 29: 2
인덱스 30: 2
인덱스 31: 2
인덱스 32: 2
인덱스 33: 2
인덱스 34: 2
인덱스 35: 2
인덱스 36: 2
인덱스 37: 2
인덱스 38: 2
인덱스 39: 2
인덱스 40: 2
인덱스 41: 2
인덱스 42: 2
인덱스 43: 2
인덱스 44: 2
인덱스 45: 2
인덱스 46: 2
인덱스 47: 2
인덱스 48: 2
인덱스 49: 2
인덱스 50: 2
인덱스 51: 2
인덱스 52: 2
인덱스 53: 2
인덱스 54: 2
인덱스 55: 2
인덱스 56: 2
인덱스 57: 2
인덱스 58: 2
인덱스 59: 2
인덱스 60: 2
인덱스 61: 2
인덱스 62: 2
인덱스 63: 2
인덱스 64: 2
인덱스 65: 2
인덱스 66: 2
인덱

----------

## 인코더 아키텍처 test

In [197]:
"""첫 번째 batch만 가져와서 확인하기"""
## 첫 번째 배치만 확인
    # 여기서 첫 번째 배치란 1열 모두를 의미한다. tensor의 shape이 (30, 128)로 되어있기 때문
for i, batch in enumerate(train_iterator):
    first_batch_src = batch.src
    first_batch_trg = batch.trg
    print(f"첫 번째 배치 크기: {first_batch_src.shape}")

    break

첫 번째 배치 크기: torch.Size([128, 37])


In [198]:
"""인코더(encoder) 객체 선언"""
INPUT_DIM = len(SRC.vocab) # 7853로 전체 단어 사전의 길이를 의미
OUTPUT_DIM = len(TRG.vocab) # 5893로 전체 번역해야 하는 단어 사전의 길이를 의미
ENCODER_EMBED_DIM = 256 # 인코더의 임베딩 차원
DECODER_EMBED_DIM = 256 # 디코더의 임베딩 차원
ENCODER_HIDDEN_DIM = 512 # 인코더 GRU의 hidden 차원
DECODER_HIDDEN_DIM = 512 # 디코더 GRU의 hidden 차원
ENC_DROPOUT_RATIO = 0.5 # 인코더 dropout 비율
DEC_DROPOUT_RATIO = 0.5 # 디코더 dropout 비율

## 인코더 입력에 대입
# enc = Encoder(INPUT_DIM=7853, ENCODER_EMBED_DIM=256, ENCODER_HIDDEN_DIM=512, DECODER_HIDDEN_DIM=512, ENC_DROPOUT_RATIO=0.5)
# enc = Encdoer(input_dim=7853, embed_dim=256,         enc_hidden_dim=512,     dec_hidden_dim=512,     dropout_ratio=0.5    )

## 첫 번째 batch에 대한 입력값 설정
input_src = first_batch_src.clone()
input_trg = first_batch_trg.clone()
print('raw 입력값 :', input_src.shape)

# 첫 번째 batch의 한 문장을 의미
print('첫 번째 batch의 한 문장 :',input_src[0])

raw 입력값 : torch.Size([128, 37])
첫 번째 batch의 한 문장 : tensor([  2,   8,  16, 184,  21,   6,  46, 107,  17,  81,  14,  94,  16,   4,
          3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1], device='cuda:0')


In [199]:
## 1. 임베딩 레이어
self_encoder_embedding = nn.Embedding(INPUT_DIM, ENCODER_EMBED_DIM).cuda()
print('embedding layer :', self_encoder_embedding)

## 임베딩 레이어
    # [batch_size, (one batch 내의) 단어 개수, embedding dimension]
embedded = self_encoder_embedding(input_src)
print('첫 batch의 임베딩 값 차원 :', embedded.shape)
print()
print('한 문장은 28개로 이루어져있고, 임베딩을 거치면 각 문장의 단어는 256차원으로 분해된다. \n 따라서 아래와 같이 (1,28)이 (28, 256)으로 변하는 것이다. \n\n ', embedded[0])

embedding layer : Embedding(7853, 256)
첫 batch의 임베딩 값 차원 : torch.Size([128, 37, 256])

한 문장은 28개로 이루어져있고, 임베딩을 거치면 각 문장의 단어는 256차원으로 분해된다. 
 따라서 아래와 같이 (1,28)이 (28, 256)으로 변하는 것이다. 

  tensor([[-0.7793,  0.4336,  0.3997,  ...,  0.0616, -1.1276,  1.4298],
        [-0.4862, -1.1893,  0.3279,  ..., -0.5035,  1.4744,  0.7188],
        [-0.5590, -0.0923, -0.4152,  ..., -0.4922,  0.1253,  1.4944],
        ...,
        [ 0.5661,  0.1912,  0.9236,  ..., -1.5110,  0.2699, -0.7312],
        [ 0.5661,  0.1912,  0.9236,  ..., -1.5110,  0.2699, -0.7312],
        [ 0.5661,  0.1912,  0.9236,  ..., -1.5110,  0.2699, -0.7312]],
       device='cuda:0', grad_fn=<SelectBackward>)


In [200]:
## 2. 인코더의 GRU 레이어
    # 임베딩된 차원을 입력으로 받아서 hidden 차원을 출력한다.
self_rnn = nn.GRU(ENCODER_EMBED_DIM, ENCODER_HIDDEN_DIM, bidirectional=True, batch_first = True).cuda()
print('rnn layer :', self_rnn)

# outputs: [batch_size, (one batch 내의) 단어 개수, encoder_hidden_dim*방향의 수] : 전체 단어의 출력 정보
# hidden: [num_layer * 방향의 수, batch_size, encoder_hidden_dim]: 현재까지의 모든 단어의 정보
output_rnn, hidden_rnn = self_rnn(embedded)
print('output :', output_rnn.shape)
print('hidden :', hidden_rnn.shape) # hidden 차원은 batch가 중간에 오게 됨
    # hidden은 [forward_layer1, backward_layer1, forward_layer2(두 번째 forward 레이어), backward_layer2(두 번째 backward 레이어), ...] 형태로 구성
    # 따라서 hidden[-2, :, :]은 forwards의 마지막 값
    # 따라서 hidden[-1, :, :]은 backwards의 마지막 값

rnn layer : GRU(256, 512, batch_first=True, bidirectional=True)
output : torch.Size([128, 37, 1024])
hidden : torch.Size([2, 128, 512])


In [201]:
## 3. 인코더의 FC 레이어
    # 인코더의 최종 hidden은 rnn의 forward hidden의 마지막 값과 backward hidden의 마지막 값을 concat하여 FC layer에 사용
    # 디코더(decoder)의 첫 번째 hidden (context) vector는 인코더의 마지막 hidden을 이용
self_fc = nn.Linear(ENCODER_HIDDEN_DIM * 2, DECODER_HIDDEN_DIM).cuda()
print('FC layer :', self_fc, end='\n\n')
print('전체 rnn의 hidden :', hidden_rnn.shape) 
print('forward rnn hidden :', hidden_rnn[-2, :, :].shape) # [batch_size, encoder_hidden_dim]
print('backward rnn hidden :', hidden_rnn[-1, :, :].shape, end='\n\n') # [batch_size, encoder_hidden_dim]

# forward layer의 최종 hidden값과 backward layer의 최종 hidden 값을 concat하여 fc layer에 넣어줌
    # 위 결과가 인코더의 최종 값이 된다.
encoder_final_hidden = torch.tanh(self_fc(torch.cat((hidden_rnn[-2,:,:], hidden_rnn[-1,:,:]), dim=1)))
print('FC layer에 입력 값 :', torch.cat((hidden_rnn[-2,:,:], hidden_rnn[-1,:,:]), dim=1).shape) # [batch_size, encoder_hidden_dim*2]

print('인코더의 첫 번째 마지막 값(RNN에서 나온 최종 결과값) :', output_rnn.shape) # [batch_size, 입력 문장 길이, rnn의 hidden feature 차원]
print('인코더의 두 번째 마지막 값(FC layer에서 나온 최종 결과값) :', encoder_final_hidden.shape) # [batch_size, decoder_hidden_dim]

FC layer : Linear(in_features=1024, out_features=512, bias=True)

전체 rnn의 hidden : torch.Size([2, 128, 512])
forward rnn hidden : torch.Size([128, 512])
backward rnn hidden : torch.Size([128, 512])

FC layer에 입력 값 : torch.Size([128, 1024])
인코더의 첫 번째 마지막 값(RNN에서 나온 최종 결과값) : torch.Size([128, 37, 1024])
인코더의 두 번째 마지막 값(FC layer에서 나온 최종 결과값) : torch.Size([128, 512])


In [202]:
"""전체 모델 구조"""
# # 어텐션(attention) 객체 선언
# attn = Attention(ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM)

# # 인코더(encoder)와 디코더(decoder) 객체 선언
# enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, ENC_DROPOUT_RATIO)
# dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, DEC_DROPOUT_RATIO, attn)

# # Seq2Seq 객체 선언
# model = Seq2Seq(enc, dec, device).to(device)

'전체 모델 구조'

----

## 디코더 & 어텐션 아키텍처 test

In [203]:
"""디코더의 입력값은 총 3개"""
## 1. <sos> 토큰으로 시작하는 값
    # batch별 문장의 <sos> 토큰 값이 들어감
## 2~3. 인코더에서 나온 결과 2개
    # 인코더 rnn output과 rnn의 hidden state를 fc layer에 넣어서 만든 값
""""디코더에 들어가는 인코더의 최종 결과"""
## output_rnn과 encoder_final_hidden
    # output_rnn, encoder_final_hidden = self.encoder(input_first_batch_src)

'"디코더에 들어가는 인코더의 최종 결과'

In [204]:
"""디코더(decoder) 객체 선언"""
INPUT_DIM = len(SRC.vocab) # 7853로 전체 단어 사전의 길이를 의미
OUTPUT_DIM = len(TRG.vocab) # 5893로 전체 번역해야 하는 단어 사전의 길이를 의미
ENCODER_EMBED_DIM = 256 # 인코더의 임베딩 차원
DECODER_EMBED_DIM = 256 # 디코더의 임베딩 차원
ENCODER_HIDDEN_DIM = 512 # 인코더 GRU의 hidden 차원
DECODER_HIDDEN_DIM = 512 # 디코더 GRU의 hidden 차원
ENC_DROPOUT_RATIO = 0.5 # 인코더 dropout 비율
DEC_DROPOUT_RATIO = 0.5 # 디코더 dropout 비율

## 디코더 입력에 대입
# enc = Decoder(OUTPUT_DIM=5893, DECODER_EMBED_DIM=256, ENCODER_HIDDEN_DIM=512, DECODER_HIDDEN_DIM=512, DEC_DROPOUT_RATIO=0.5, attn)
# enc = Decdoer(output_dim=5893, embed_dim=256,         enc_hidden_dim=512,     dec_hidden_dim=512,     dropout_ratio=0.5,     attention(클래스))

## 첫 번째 batch에 대한 입력값 설정
input_src = first_batch_src.clone()
input_trg = first_batch_trg.clone()

In [205]:
"""1-1. 디코더 입력값 설정"""
## 인코더의 최종 hidden과 인코더의 최종 output을 비교하는 게 attention의 첫 시작
    # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
    # hidden: [배치 크기, 히든 차원] (인코더의 최종 hidden)
    # enc_outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보

## 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
    # input_trg : [batch_size, 번역해야 하는 단어의 길이]
trg_len = input_trg.shape[1]
print('첫 번째 batch에 대한 번역 문장의 단어 개수 :', trg_len, end='\n\n')

    # 배치 크기
batch_size = input_trg.shape[0]
print('batch size :', batch_size, end='\n\n')

    # 출력 차원
        # 출력해야하는 trg에 대한 단어 사전 임베딩 차원
        # 5893 -> 256
trg_vocab_size = OUTPUT_DIM 
print('출력 차원 :', trg_vocab_size, end='\n\n')
    
    # 디코더의 최종 결과를 담을 텐서 객체
        # [batch_size, 한 배치에 대한 문장의 단어 개수, 출력하는 임베딩 차원]
outputs_decoder = torch.zeros(batch_size, trg_len, trg_vocab_size)
print('디코더의 최종 결과를 담을 텐서 객체 :', outputs_decoder.shape , end='\n\n')


## batch별 첫 번째 입력은 항상 <sos> 토큰
input_decoder = input_trg[:,0]
print('디코더 첫 입력값 <sos> 토큰 :', input_decoder)
print(input_decoder.shape)

첫 번째 batch에 대한 번역 문장의 단어 개수 : 36

batch size : 128

출력 차원 : 5893

디코더의 최종 결과를 담을 텐서 객체 : torch.Size([128, 36, 5893])

디코더 첫 입력값 <sos> 토큰 : tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
torch.Size([128])


In [206]:
"""1-2. 디코더 입력값 설정"""
## 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환
## self.decoder(input, hidden, enc_outputs)
    # input = [1개 단어] : 단어의 개수는 항상 1개이도록 구현
    # hidden = [batch_size, 인코더 hidden feature 차원] : 인코더의 최종 fc layer를 거친 hidden 값
    # enc_outputs = [batch_size, 단어 개수, 인코더 hidden 차원*방향의 수] : 인코더 RNN의 출력값
input_decoder_ = input_decoder.unsqueeze(1)
print(input_decoder_.shape, end='\n\n') # [배치 크기, 단어 개수]

torch.Size([128, 1])



In [207]:
"""2. 디코더 임베딩 레이어"""
decoder_self_embedding = nn.Embedding(OUTPUT_DIM, DECODER_EMBED_DIM).to(device) # 디코더에서 번역해야하는 단어 목록을 임베딩
decoder_embedded = decoder_self_embedding(input_decoder_)
print('임베딩된 디코더 값:', decoder_embedded.shape)

임베딩된 디코더 값: torch.Size([128, 1, 256])


In [208]:
"""3. 어텐션 아키텍처"""
batch_size = output_rnn.shape[0]
src_len = output_rnn.shape[1] # 단어 길이

## 현재 디코더의 hidden state를 src_len만큼 반복
    # 여기서 현재 디코더의 hidden_state의 '첫번째 값'은 인코더으 최종 hidden state를 의미하고, 이후 forwarding이 진행될 때 업데이트된다.
hidden_att = encoder_final_hidden.unsqueeze(1).repeat(1, src_len, 1)
enc_outputs = output_rnn # 위의 코드에서는 batch_first=False이기에 .permute(1,0,2)를 해주어야 한다.
print('현재의 디코더 hidden값 :', hidden_att.shape) # hidden_att : [배치 크기, 단어 개수, 디코더 히든 차원]: 현재까지의 모든 단어의 정보
print('현재의 인코더 output 값 :', enc_outputs.shape) # enc_outputs : [배치 크기, 단어 개수, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
print()

## 어텐션 에너지 만들기
    # 현재의 디코더 hidden 값과 인코더 최종 output 값의 feature concat
energy_input = torch.cat((hidden_att, enc_outputs), dim=2)
print('어텐션 에너지 입력값 :', energy_input.shape)

self_attn = nn.Linear((ENCODER_HIDDEN_DIM * 2) + DECODER_HIDDEN_DIM, DECODER_HIDDEN_DIM).to(device)
# print('어텐션 레이어 :', self_attn)

energy = self_attn(energy_input) # energy: [배치 크기, 단어 개수, 디코더 히든 차원]
print('에너지 차원 :', energy.shape) 

self_v = nn.Linear(DECODER_HIDDEN_DIM, 1, bias=False).to(device) # attention: [배치 크기, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들
attention = self_v(energy).squeeze(2)
print('소프트맥스 전의 최종 어텐션 값 :', attention.shape) # 이를 통해 각 문장의 단어 28개에 대한 가중치를 구한다.
print('소프트맥스 후의 어텐션 값 :', F.softmax(attention, dim=1).shape)

현재의 디코더 hidden값 : torch.Size([128, 37, 512])
현재의 인코더 output 값 : torch.Size([128, 37, 1024])

어텐션 에너지 입력값 : torch.Size([128, 37, 1536])
에너지 차원 : torch.Size([128, 37, 512])
소프트맥스 전의 최종 어텐션 값 : torch.Size([128, 37])
소프트맥스 후의 어텐션 값 : torch.Size([128, 37])


In [209]:
"""4. 어텐션 weight 반영"""
attention = attention.unsqueeze(1) #  attention: [배치 크기, 1, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들
print('실제 각 단어에 대한 attention 값들 :', attention.shape, end='\n\n')

enc_outputs = output_rnn # [배치 크기, 단어 개수, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
print('인코더 rnn 출력값 :', enc_outputs.shape, end='\n\n')

weighted = torch.bmm(attention, enc_outputs) # 행렬곱 함수
# (batch_size, 1, 28) * (batch_size, 28, 1024)
print('weighted shape :', weighted.shape) # [배치 크기, 1, 인코더 히든 차원 * 방향의 수]

실제 각 단어에 대한 attention 값들 : torch.Size([128, 1, 37])

인코더 rnn 출력값 : torch.Size([128, 37, 1024])

weighted shape : torch.Size([128, 1, 1024])


In [210]:
"""5. 디코더 임베딩과 어텐션 weight 값의 feature들 concat"""
decoder_rnn_input = torch.cat((decoder_embedded, weighted), dim=2) # rnn_input: [1, 배치 크기, 인코더 히든 차원 * 방향의 수 + embed_dim]: 어텐션이 적용된 현재 단어 입력 정보
print('디코더 rnn의 입력값 :', decoder_rnn_input.shape)

디코더 rnn의 입력값 : torch.Size([128, 1, 1280])


In [211]:
"""6. 디코더의 출력값 생성 (rnn 사용)"""
self_rnn = nn.GRU((ENCODER_HIDDEN_DIM * 2) + DECODER_EMBED_DIM, DECODER_HIDDEN_DIM, batch_first=True).to(device)
output, hidden = self_rnn(decoder_rnn_input, encoder_final_hidden.unsqueeze(0))
print('디코더의 rnn 결과 :', output.shape) # output: [단어 개수, 배치 크기, 디코더 히든 차원 * 방향의 수]
print('디코더의 hidden 값 결과 :',hidden.shape) # hidden: [레이어 개수 * 방향의 수, 배치 크기, 디코더 히든 차원]: 현재까지의 모든 단어의 정보

## 출력은 단 하나의 값
    # 현재 예제에서는 단어 개수, 레이어 개수, 방향의 수 모두 1의 값을 가짐
    # 따라서 output: [1, 배치 크기, 디코더 히든 차원], hidden: [1, 배치 크기, 디코더 히든 차원]
    # 다시 말해 output과 hidden의 값 또한 동일

디코더의 rnn 결과 : torch.Size([128, 1, 512])
디코더의 hidden 값 결과 : torch.Size([1, 128, 512])


In [217]:
"""최종 FC layer"""
decoder_embedded_ = decoder_embedded.squeeze(1)
output_ = output.squeeze(1) # 디코더 rnn의 결과
weighted_ = weighted.squeeze(1) # 어텐션 weight
print(decoder_embedded_.shape)
print(output_.shape)
print(weighted_.shape)

torch.Size([128, 256])
torch.Size([128, 512])
torch.Size([128, 1024])


In [219]:
self_fc_out = nn.Linear((ENCODER_HIDDEN_DIM * 2) + DECODER_HIDDEN_DIM + DECODER_EMBED_DIM, OUTPUT_DIM).to(device)
print('최종 FC layer :', self_fc_out, end='\n\n')
prediction = self_fc_out(torch.cat((output_, weighted_, decoder_embedded_), dim=1))
print('최종 예측값 :', prediction.shape)

최종 FC layer : Linear(in_features=1792, out_features=5893, bias=True)

최종 예측값 : torch.Size([128, 5893])


In [156]:
## 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
# for t in range(1, trg_len):
#     output, hidden = self.decoder(input, hidden, enc_outputs) # 여기서 hidden은 인코더 hidden

#     outputs[t] = output # FC를 거쳐서 나온 현재의 출력 단어 정보
#     top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

#     # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
#     teacher_force = random.random() < teacher_forcing_ratio
#     input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기